# Cargando las librerias requeridas

In [ ]:
import numpy as np
import pandas as pd
from plotnine import *

In [ ]:
from sklearn import datasets
iris = datasets.load_iris() #Cargando el dataset de vino
print(iris.keys()) #Analizando las variables que tiene

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_features = scaler.fit_transform(iris.data) #Transformamos los datos

# Cohesion y separación
Los coeficientes de cohesión y separación deben ser calculados por nosotros mismos. Por lo cual, usando el código de abajo podemos calcularlos en forma rápida.

In [ ]:
#Calculando la cohesión y separación
from sklearn.cluster import KMeans
km = KMeans(n_clusters=2)
km=km.fit(scaled_features)

tempCohesion=0
for j in range(km.n_clusters):
    #Calculando el centro del clusters
    #centroCluster=np.mean(scaled_features[km.labels_==j,:],axis=0)
    centroCluster=km.cluster_centers_[j,:]
    #Calculando la cohesion para ese cluster
    tempCohesion+=np.square(scaled_features[km.labels_==j,:]-centroCluster).sum()
print("Cohesión: ",tempCohesion)
print("Inertia: ",km.inertia_) #For k-means

tempSeparacion=0
#Calculando el centro de los datos
centroData=np.mean(scaled_features,axis=0)
for j in range(km.n_clusters):
    #Calculando el centro del clusters
    #centroCluster=np.mean(scaled_features[km.labels_==j,:],axis=0)
    centroCluster=km.cluster_centers_[j,:]
    #Calculando la separación para ese cluster
    tempSeparacion+=(km.labels_==j).sum()*np.square(centroCluster-centroData).sum()
print("Separación: ",tempSeparacion)
print("Separación + cohesión: ",tempSeparacion+tempCohesion)



# Coeficiente de silhouette 

El coeficiente de silhouette promedio se calcula con la función silhouette_score del paquete sklearn.metrics.<br>
silhouette_score(X, labels, metric='euclidean', sample_size=None, random_state=None)<br><br>
Parámetros
* X: set de datos o matriz de distancia.
* labels: etiquetas de los clusters.
* metric: métrica de distancia a utilizar, si X es una matriz de distancia se utiliza "precomputed".
* sample_size: número de puntos (para grandes bases de datos).
* random_state: inicialización del número aleatorio para el sampleo.<br><br>

De igual manera, la función silhouette_samples calcula el coeficiente de Silhouette para cada punto por separado<br>
silhouette_samples(X, labels, metric='euclidean')
<br><br>
Parámetros
* X: set de datos o matriz de distancia.
* labels: etiquetas de los clusters.
* metric: métrica de distancia a utilizar, si X es una matriz de distancia se utiliza "precomputed".

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
numClusters=3
km = KMeans(n_clusters=numClusters)
km=km.fit(scaled_features)

#Calculando el coeficiente de Silhouette para cada muestra
sample_silhouette_values = silhouette_samples(scaled_features, km.labels_)
print(sample_silhouette_values)

#Calculando el coeficiente de silhouette promedio
silhouette_avg = silhouette_score(scaled_features, km.labels_)
print("Para n_clusters =", numClusters,"el coeficiente promedio de silhouette_score es:", silhouette_avg)
print("Equivalente a ",np.average(silhouette_samples(scaled_features, km.labels_)))


In [ ]:
#Visualizando los coeficientes de Silhouette
tempDF = pd.DataFrame(columns=['SC','labels'])
for i in range(km.n_clusters):
    #Ordenando los valores de cada cluster
    tempData=np.sort(sample_silhouette_values[km.labels_==i])[::-1]#para mostrarlos de menor a mayor
    tempDF2=pd.DataFrame(tempData,columns=["SC"])
    tempDF2["labels"]=i
    #Agregandolo a un nuevo data frame
    tempDF=tempDF.append(tempDF2)
tempDF.reset_index()
(ggplot(tempDF)+aes(y="SC",x=range(150),color="factor(labels)")
 +geom_col(show_legend=False)+coord_flip()+theme_bw()
 +labs(y="Coeficiente de Silhouette")+theme(axis_text_y=element_blank(),axis_ticks_major_y=element_blank())
)

# Buscando el mejor número de clusters

In [ ]:
from sklearn.metrics import silhouette_samples, silhouette_score
maxK=11
sse = np.zeros(maxK-2) # NO USAR []
CS = np.zeros(maxK-2) # NO USAR []
for k in range(2, maxK):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(scaled_features)
    sse[k-2]=kmeans.inertia_ #NO USAR sse.append(kmeans.inertia_)
    silhouette_avg = silhouette_score(scaled_features, kmeans.labels_)
    CS[k-2]=silhouette_avg #NO USAR CS.append(silhouette_avg)
    

tempData=pd.DataFrame(range(2, maxK),columns=["K"])
tempData["sse"]=sse
tempData["CS"]=CS
print(ggplot(tempData)+aes(x="K",y="sse")+theme_bw()+geom_line()+labs(x="Número de clusters",y="WCD")+scale_x_continuous(breaks=range(1,maxK)))
print(ggplot(tempData)+aes(x="K",y="CS")+theme_bw()+geom_line()+labs(x="Número de clusters",y="Coeficiente de Silhouette promedio")+scale_x_continuous(breaks=range(1,maxK)))    


# Hopkins statistics
La función hopkins del paquete pyclustertend permite calcular la tendencia de hopkins en forma rápida. Sin embargo, tiene problemas con largos set de datos.<br>

La función recibe solo dos parámetros, los datos y el número de puntos aleatorios a generar.<br>
hopkins(dataset,numSample)

Recuerde calcula 1 menos el valor del test

In [ ]:
#Importando la función de hopkins
from pyclustertend import hopkins
print("Coeficiente de Hopkins, datos de iris escalados:",1-hopkins(scaled_features,50))

#Generando 150 datos aleatorios
tempData=np.random.uniform(low=0,high=1,size=(150,2))
tempData=pd.DataFrame(tempData,columns=["X1","X2"])
print(ggplot(tempData)+aes(x="X1",y="X2")+geom_point())
print("Coeficiente de Hopkins, datos aleatorios:",1-hopkins(tempData,30))



# Evaluación matriz de distancia
Para evaluar la matriz de distancia entre los puntos, utilizaremos las funciones vat del paquete pyclustertend. Tal como en el caso anterior, se tiene problemas con largos set de datos.<br>

La función recibe un set de datos y despliega la matriz de distancia en forma ordenada para mejorar su visualización.


In [ ]:
#Instalando el modulo requerido
#import sys
#!{sys.executable} -m pip install pyclustertend

In [ ]:
from pyclustertend import vat
vat(scaled_features)
vat(tempData)